In [ ]:
#Import libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [ ]:
#Load the dataset
#from google.colab import files
#uploaded = files.upload()
dfs = [pd.read_csv('user_'+ user +'.csv') for user in ['a','b','c','d']]

In [ ]:
#Adds a column to the dfs to indentify the user
for i in range (len(dfs)):
  dfs[i]['User'] = pd.Series(i, index=dfs[i].index)

In [ ]:
#Concat all the dfs into a single df, shuffle the data, and reset index
#axis 0 is the row (y) axis. This ensures that they are stacked on top of each other rather than side by side
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)
data

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,F7 theta std,F7 theta m,F7 alpha std,F7 alpha m,F7 beta std,F7 beta m,F3 delta std,F3 delta m,F3 theta std,F3 theta m,F3 alpha std,F3 alpha m,F3 beta std,F3 beta m,FC5 delta std,FC5 delta m,FC5 theta std,FC5 theta m,FC5 alpha std,FC5 alpha m,FC5 beta std,FC5 beta m,T7 delta std,T7 delta m,T7 theta std,T7 theta m,T7 alpha std,T7 alpha m,T7 beta std,...,T8 delta m,T8 theta std,T8 theta m,T8 alpha std,T8 alpha m,T8 beta std,T8 beta m,FC6 delta std,FC6 delta m,FC6 theta std,FC6 theta m,FC6 alpha std,FC6 alpha m,FC6 beta std,FC6 beta m,F4 delta std,F4 delta m,F4 theta std,F4 theta m,F4 alpha std,F4 alpha m,F4 beta std,F4 beta m,F8 delta std,F8 delta_m,F8 theta std,F8 theta m,F8 alpha std,F8 alpha m,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,2061.195445,0.262423,1.902286,0.733497,1.603750,1.535570,2.179065,3628.046889,2117.275756,5.735811,21.121379,8.769352,33.547213,46.439133,47.406791,3562.345232,2057.952040,0.397706,1.844876,0.846321,2.437813,0.842549,1.917293,3561.120526,2058.586386,0.257960,1.905127,0.430142,2.621501,1.138976,...,2088.219247,3.294524,11.875649,3.719521,16.050149,28.043325,28.407129,3622.155968,2124.806117,5.522155,21.561988,3.604795,30.540629,42.432765,43.521641,3542.731860,2048.032429,0.699359,2.798018,1.047519,3.834498,4.884267,5.510267,3646.338310,2121.894637,6.241312,14.722455,4.325903,22.016671,35.417275,35.151586,3628.426885,2129.789645,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,2062.290606,1.448238,1.644273,1.173074,2.982328,1.123690,1.971145,3649.554135,2146.893843,10.783758,39.055984,14.820892,26.983965,25.876203,54.351914,3561.881351,2058.629466,0.790864,1.334296,0.538223,1.590056,0.710888,1.676324,3557.993216,2055.787289,1.891363,2.336553,1.382140,2.290540,0.784874,...,2095.929658,4.890093,19.746521,7.518262,14.380510,15.161397,32.792210,3627.557950,2128.870783,10.228892,37.023010,11.567969,23.857091,24.594590,51.831668,3556.389426,2060.587846,1.398379,3.815512,0.931655,3.609845,3.375365,7.288658,3661.990245,2135.983652,10.260750,26.597810,8.858497,19.609274,19.403818,40.800889,3680.341349,2144.200503,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,2061.818772,1.193787,2.737070,1.141789,2.678112,1.636123,3.230015,3614.068576,2253.930544,13.171740,57.354226,4.858445,30.813375,5.202954,16.917773,3528.564741,2065.374551,4.403050,8.825202,0.644223,6.445244,1.418807,3.530294,3477.533537,2096.952007,7.335056,29.406115,3.040467,15.960430,3.100798,...,2062.191733,3.937643,11.656560,0.822316,4.896451,1.114029,2.675689,3480.463194,2060.693476,5.259307,19.035779,1.911211,11.966643,2.183745,5.895479,3555.992474,2165.927106,9.883313,41.794649,3.508704,23.279279,4.301297,12.370112,3732.299691,2330.652224,14.642631,57.885099,4.216069,32.983829,5.672901,18.176841,3538.347368,2081.315814,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,2
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,2060.271106,0.349459,2.199150,1.305408,3.477799,1.391512,1.941253,3585.392270,2103.011304,16.990012,16.921309,23.845017,51.808441,34.559939,74.654695,3564.326621,2059.822694,0.433203,1.399931,0.861600,2.524651,1.006103,2.069075,3556.186742,2055.942550,0.644386,1.348078,1.056468,2.858424,1.294326,...,2081.810111,8.247690,10.227615,11.924389,27.059693,19.846525,42.964105,3578.686049,2091.516282,14.398442,16.247489,18.718327,42.063033,29.243286,62.505560,3575.462654,2067.132185,0.904821,2.214364,1.084846,3.418401,1.628869,3.451918,3603.515363,2098.932571,11.6

Helper Functions

In [ ]:
def onehot_encode(df, column):
  df = df.copy()
  dummies = pd.get_dummies(df[column], prefix=column)
  df = pd.concat([df, dummies], axis=1)
  df = df.drop(column, axis = 1)
  return df

In [ ]:
def preprocess_inputs(df, target='Class'):
    df = df.copy()
    
    # One-hot encode whichever target column is not being used
    targets = ['Class', 'User']
    targets.remove(target)
    df = onehot_encode(df, column=targets[0])
    
    # Split df into X and y
    y = df[target].copy()
    X = df.drop(target, axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def build_model(num_classes=3):
  inputs = tf.keras.Input(shape=(X_train.shape[1],))
  x = tf.keras.layers.Dense(128, activation='relu')(inputs)
  x = tf.keras.layers.Dense(128, activation='relu')(x)
  outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
      optimizer = 'adam',
      loss='sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )

  return model

Predecting Hand Movement Class

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='Class')

In [ ]:
class_model = build_model(num_classes=3)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(
                   monitor='val_loss',
                   patience=3,
                   restore_best_weights = True
               )
    ]
)

Epoch 1/50
202/202 [==============================] - 1s 3ms/step - loss: 1.1067 - accuracy: 0.3984 - val_loss: 1.0262 - val_accuracy: 0.4693
Epoch 2/50
202/202 [==============================] - 0s 2ms/step - loss: 0.9943 - accuracy: 0.4899 - val_loss: 1.0078 - val_accuracy: 0.4997
Epoch 3/50
202/202 [==============================] - 0s 2ms/step - loss: 0.9342 - accuracy: 0.5378 - val_loss: 0.9618 - val_accuracy: 0.5251
Epoch 4/50
202/202 [==============================] - 0s 2ms/step - loss: 0.8837 - accuracy: 0.5815 - val_loss: 0.9291 - val_accuracy: 0.5425
Epoch 5/50
202/202 [==============================] - 0s 2ms/step - loss: 0.8414 - accuracy: 0.6067 - val_loss: 0.9126 - val_accuracy: 0.5598
Epoch 6/50
202/202 [==============================] - 0s 2ms/step - loss: 0.7983 - accuracy: 0.6323 - val_loss: 0.9110 - val_accuracy: 0.5592
Epoch 7/50
202/202 [==============================] - 0s 2ms/step - loss: 0.7457 - accuracy: 0.6733 - val_loss: 0.8981 - val_accuracy: 0.5704
Epoch 

In [ ]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print('Test Accuracy (Class Model): {:.2f}%'.format(class_acc * 100))

Test Accuracy (Class Model): 61.70%


Predecting User

In [ ]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='User')

In [ ]:
user_model = build_model(num_classes=4)

user_history = user_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(
                   monitor='val_loss',
                   patience=3,
                   restore_best_weights = True
               )
    ]
)

Epoch 1/50
202/202 [==============================] - 1s 3ms/step - loss: 0.3302 - accuracy: 0.8850 - val_loss: 0.0288 - val_accuracy: 0.9907
Epoch 2/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0202 - accuracy: 0.9927 - val_loss: 0.0131 - val_accuracy: 0.9950
Epoch 3/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0093 - accuracy: 0.9966 - val_loss: 0.0054 - val_accuracy: 0.9988
Epoch 4/50
202/202 [==============================] - 1s 3ms/step - loss: 0.0031 - accuracy: 0.9995 - val_loss: 0.0049 - val_accuracy: 0.9981
Epoch 5/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0054 - val_accuracy: 0.9975
Epoch 6/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0036 - val_accuracy: 0.9981
Epoch 7/50
202/202 [==============================] - 0s 2ms/step - loss: 6.8160e-04 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 0.9988
Ep

In [ ]:
user_acc = user_model.evaluate(X_test, y_test, verbose=0)[1]
print('Test Accuracy (User Model): {:.2f}%'.format(user_acc * 100))

Test Accuracy (User Model): 99.94%
